<p align = "center" draggable=”false” ><img src="https://user-images.githubusercontent.com/37101144/161836199-fdb0219d-0361-4988-bf26-48b0fad160a3.png" 
     width="200px"
     height="auto"/>
</p>

# <h1 align="center" id="heading">Sentiment Analysis of Reddit Data using Reddit API</h1>

In this live coding session, we leverage the Python Reddit API Wrapper (`PRAW`) to retrieve data from subreddits on [Reddit](https://www.reddit.com), and perform sentiment analysis using [`pipelines`](https://huggingface.co/docs/transformers/main_classes/pipelines) from [HuggingFace ( 🤗 the GitHub of Machine Learning )](https://techcrunch.com/2022/05/09/hugging-face-reaches-2-billion-valuation-to-build-the-github-of-machine-learning/), powered by [transformer](https://arxiv.org/pdf/1706.03762.pdf).

## Objectives

At the end of the session, you will 

- know how to work with APIs
- feel more comfortable navigating thru documentation, even inspecting the source code
- understand what a `pipeline` object is in HuggingFace
- perform sentiment analysis using `pipeline`
- run a python script in command line and get the results

## How to Submit

- At the end of each task, commit* the work into the repository you created before the assignment
- After completing all three tasks, make sure to push the notebook containing all code blocks and output cells to your repository you created before the assignment
- Submit the link to the notebook in Canvas

\***NEVER** commit a notebook displaying errors unless it is instructed otherwise. However, commit often; recall git ABC = **A**lways **B**e **C**ommitting.

## Tasks

### Task I: Instantiate a Reddit API Object

The first task is to instantiate a Reddit API object using [PRAW](https://praw.readthedocs.io/en/stable/), through which you will retrieve data. PRAW is a wrapper for [Reddit API](https://www.reddit.com/dev/api) that makes interacting with the Reddit API easier unless you are already an expert of [`requests`](https://docs.python-requests.org/en/latest/).

#### 1. Install packages

Please ensure you've ran all the cells in the `imports.ipynb`, located [here](https://github.com/FourthBrain/MLE-8/blob/main/assignments/week-3-analyze-sentiment-subreddit/imports.ipynb), to make sure you have all the required packages for today's assignment.

####  2. Create a new app on Reddit 

Create a new app on Reddit and save secret tokens; refer to [post in medium](https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c) for more details.

- Create a Reddit account if you don't have one, log into your account.
- To access the API, we need create an app. Slight updates, on the website, you need to navigate to `preference` > `app`, or click [this link](https://www.reddit.com/prefs/apps) and scroll all the way down. 
- Click to create a new app, fill in the **name**, choose `script`, fill in  **description** and **redirect uri** ( The redirect URI is where the user is sent after they've granted OAuth access to your application (more info [here](https://github.com/reddit-archive/reddit/wiki/OAuth2)) For our purpose, you can enter some random url, e.g., www.google.com; as shown below.


    <img src="https://miro.medium.com/max/700/1*lRBvxpIe8J2nZYJ6ucMgHA.png" width="500"/>
- Jot down `client_id` (left upper corner) and `client_secret` 

    NOTE: CLIENT_ID refers to 'personal use script" and CLIENT_SECRET to secret.
    
    <div>
    <img src="https://miro.medium.com/max/700/1*7cGAKth1PMrEf2sHcQWPoA.png" width="300"/>
    </div>

- Create `secrets.py` in the same directory with this notebook, fill in `client_id` and `secret_id` obtained from the last step. We will need to import those constants in the next step.
    ```
    REDDIT_API_CLIENT_ID = "client_id"
    REDDIT_API_CLIENT_SECRET = "secret_id"
    REDDIT_API_USER_AGENT = "any string except bot; ex. My User Agent"
    ```
- Add `secrets.py` to your `.gitignore` file if not already done. NEVER push credentials to a repo, private or public. 

#### 3. Instantiate a `Reddit` object

Now you are ready to create a read-only `Reddit` instance. Refer to [documentation](https://praw.readthedocs.io/en/stable/code_overview/reddit_instance.html) when necessary.

In [41]:
import praw
import reddit_secrets

# Create a Reddit object which allows us to interact with the Reddit API
reddit = praw.Reddit(
    client_id=reddit_secrets.REDDIT_API_CLIENT_ID,
    client_secret=reddit_secrets.REDDIT_API_CLIENT_SECRET,
    user_agent=reddit_secrets.REDDIT_API_USER_AGENT,
)

In [42]:
print(reddit) 

<details>
<summary>Expected output:</summary>   

```<praw.reddit.Reddit object at 0x10f8a0ac0>```
</details>

#### 4. Instantiate a `subreddit` object

Lastly, create a `subreddit` object for your favorite subreddit and inspect the object. The expected output you will see ar from `r/machinelearning` unless otherwise specified.

In [43]:
subreddit = reddit.subreddit("machinelearning")

What is the display name of the subreddit?

In [44]:
print(subreddit.display_name)

machinelearning


<details>
<summary>Expected output:</summary>   

    machinelearning
</details>

How about its title, is it different from the display name?

In [45]:
print(subreddit.title)

Machine Learning


<details>
<summary>Expected output:</summary>   

    Machine Learning
</details>

Print out the description of the subreddit:

In [46]:
print(subreddit.description[:500])

**[Rules For Posts](https://www.reddit.com/r/MachineLearning/about/rules/)**
--------
+[Research](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AResearch)
--------
+[Discussion](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3ADiscussion)
--------
+[Project](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AProject)
--------
+[News](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict


<details>
<summary>Expected output:</summary>

    **[Rules For Posts](https://www.reddit.com/r/MachineLearning/about/rules/)**
    --------
    +[Research](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AResearch)
    --------
    +[Discussion](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3ADiscussion)
    --------
    +[Project](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AProject)
    --------
    +[News](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict
</details>

### Task II: Parse comments

#### 1. Top Posts of All Time

Find titles of top 10 posts of **all time** from your favorite subreddit. Refer to [Obtain Submission Instances from a Subreddit Section](https://praw.readthedocs.io/en/stable/getting_started/quick_start.html)) if necessary. Verify if the titles match what you read on Reddit.

In [48]:
# try running this line, what do you see? press q once you are done
?subreddit.top 

In [49]:
for submission in subreddit.top(limit=10):
    print(submission.title)

[Project] From books to presentations in 10s with AR + ML
[D] A Demo from 1993 of 32-year-old Yann LeCun showing off the World's first Convolutional Network for Text Recognition
[R] First Order Motion Model applied to animate paintings
[N] AI can turn old photos into moving Images / Link is given in the comments - You can also turn your old photo like this
[D] This AI reveals how much time politicians stare at their phone at work
[D] Types of Machine Learning Papers
[D] The machine learning community has a toxicity problem
[Project] NEW PYTHON PACKAGE: Sync GAN Art to Music with "Lucid Sonic Dreams"! (Link in Comments)
I made a robot that punishes me if it detects that if I am procrastinating on my assignments [P]
[P] Using oil portraits and First Order Model to bring the paintings back to life


<details> <summary>Expected output:</summary>

    [Project] From books to presentations in 10s with AR + ML
    [D] A Demo from 1993 of 32-year-old Yann LeCun showing off the World's first Convolutional Network for Text Recognition
    [R] First Order Motion Model applied to animate paintings
    [N] AI can turn old photos into moving Images / Link is given in the comments - You can also turn your old photo like this
    [D] This AI reveals how much time politicians stare at their phone at work
    [D] Types of Machine Learning Papers
    [D] The machine learning community has a toxicity problem
    [Project] NEW PYTHON PACKAGE: Sync GAN Art to Music with "Lucid Sonic Dreams"! (Link in Comments)
    [P] Using oil portraits and First Order Model to bring the paintings back to life
    [D] Convolution Neural Network Visualization - Made with Unity 3D and lots of Code / source - stefsietz (IG)    
</details>

#### 2. Top 10 Posts of This Week

What are the titles of the top 10 posts of **this week** from your favorite subreddit?

In [50]:
for submission in subreddit.top(time_filter="week", limit=10):
    print(submission.title)

30% of Google's Reddit Emotions Dataset is Mislabeled [D]
[R] mixed reality future — see the world through artistic lenses — made with NeRF
[N] First-Ever Course on Transformers: NOW PUBLIC
[D] Why are Corgi dogs so popular in machine learning (especially in the image generation community)?
[D] Are there any rejected papers that ended up having significant impact in the long run?
[D] Noam Chomsky on LLMs and discussion of LeCun paper (MLST)
[N] Andrej Karpathy is leaving Tesla
[R] So someone actually peer-reviewed this and thought "yeah, looks good"?
[D] How do you verify the novelty of your research?
[N] BigScience Releases their 176 Billion Parameter Open-access Multilingual Language Model


<details><summary>Expected output:</summary>

    [N] Ian Goodfellow, Apple’s director of machine learning, is leaving the company due to its return to work policy. In a note to staff, he said “I believe strongly that more flexibility would have been the best policy for my team.” He was likely the company’s most cited ML expert.
    [R][P] Thin-Plate Spline Motion Model for Image Animation + Gradio Web Demo
    [P] I’ve been trying to understand the limits of some of the available machine learning models out there. Built an app that lets you try a mix of CLIP from Open AI + Apple’s version of MobileNet, and more directly on your phone's camera roll.
    [R] Meta is releasing a 175B parameter language model
    [N] Hugging Face raised $100M at $2B to double down on community, open-source & ethics
    [P] T-SNE to view and order your Spotify tracks
    [D] : HELP Finding a Book - A book written for Google Engineers about foundational Math to support ML
    [R] Scaled up CLIP-like model (~2B) shows 86% Zero-shot on Imagenet
    [D] Do you use NLTK or Spacy for text preprocessing?
    [D] Democratizing Diffusion Models - LDMs: High-Resolution Image Synthesis with Latent Diffusion Models, a 5-minute paper summary by Casual GAN Papers
</details>

💽❓ Data Question:

Check out what other attributes the `praw.models.Submission` class has in the [docs](https://praw.readthedocs.io/en/stable/code_overview/models/submission.html). 

1. After having a chance to look through the docs, is there any other information that you might want to extract? How might this additional data help you?

Write a sample piece of code below extracting three additional pieces of information from the submission below.

ANSWER: The text of the comment (body), the number of comments (num_comments), and the number of upvotes for the comment (score) are all important enough to extract. Having the text of the comment would make it easy to do a sanity check on a task like sentiment analysis and also examine any issues with the data, like bias or irrelevance to the topic at hand. Having the number of comments replying to this comment is an indication of its overall importance to the original post, as measured by the level of community feedback. The score of the comment would be an important indication of its relevance to the post as well as overall sentiment, since overly aggressive and unwarranted comments are often downvoted depending on the subreddit and would therefore have a low score.

In [51]:
submission = reddit.submission("she44y")
for comment in submission.comments:
    print(comment.body)
print("Number of comments: ", submission.num_comments)
print("Score: ", submission.score)

I asked a similar question 10 years ago: https://old.reddit.com/r/redditdev/comments/n624n/submission_ids_question/
I don't think there's a "why" unless you happen to catch u/ ketralnis' attention and he remembers the history. They just made changes to how ids worked back then. There have been other, much larger gaps in the ids since then too. They just need to do database work or something and decide to skip some ids.
You'd have to ask the developers themselves, but it's probably a simple matter of the early site being in constant development and change. The gaps would be explained by large scale internal testing or more simply by bugs.
Number of comments:  3
Score:  7


💽❓ Data Question:

2. Is there any information available that might be a concern when it comes to Ethical Data?

ANSWER: The author of the submission as well as the time the submission was created (created_utc) could both be a concern when it comes to Ethical Data as they can both potentially be used to figure out personal information about the poster. The author of the submission is typically unimportant when doing ML tasks related to the text of the posts and comments, and although it's public information, it would still be good to take initiative and hide that data from being shared outside of Reddit. Also, the time the submission was created could be an indication of the time zone the poster lives in.

#### 3. Comment Code

Add comments to the code block below to describe what each line of the code does (Refer to [Obtain Comment Instances Section](https://praw.readthedocs.io/en/stable/getting_started/quick_start.html) when necessary). The code is adapted from [this tutorial](https://praw.readthedocs.io/en/stable/tutorials/comments.html)

The purpose is 
1. to understand what the code is doing 
2. start to comment your code whenever it is not self-explantory if you have not (others will thank you, YOU will thank you later 😊) 

In [53]:
%%time
from praw.models import MoreComments

# Initiate top comments list which will hold comments from 10 top posts
top_comments = []

# Iteration through 10 top posts of all time in this subreddit
for submission in subreddit.top(limit=10):
    # Iterate through top-level comments in the current post
    for top_level_comment in submission.comments:
        # Identify if this comment has more replies
        if isinstance(top_level_comment, MoreComments):
            continue
        # Add current comment to top comment list
        top_comments.append(top_level_comment.body)

CPU times: user 339 ms, sys: 349 ms, total: 687 ms
Wall time: 31.6 s


#### 4. Inspect Comments

How many comments did you extract from the last step? Examine a few comments. 

In [54]:
# the answer may vary 693 for r/machinelearning
print(len(top_comments))

740


In [56]:
import random

[random.choice(top_comments) for i in range(3)]

['Couldn’t stop watching Nefertiti. What a babe!',
 'This is what tripping is like :D',
 'In 3 seconds if you use anything else then MacBook 😏']

<details> <summary>Some of the comments from `r/machinelearning` subreddit are:</summary>

    ['Awesome visualisation',
    'Similar to a stack or connected neurons.',
    'Will this Turing pass the Turing Test?']
</details>

💽❓ Data Question:

3. After having a chance to review a few samples of 5 comments from the subreddit, what can you say about the data? 

HINT: Think about the "cleanliness" of the data, the content of the data, think about what you're trying to do - how does this data line up with your goal?

ANSWER: This data is not very clean, particularly in the sense that there is a wide range of comments: some relevant and helpful, some funny, some totally irrelevant. Even if the task we were working on involved the machine learning subreddit and getting a sense of the machine learning community from random comments, it is clear from the first two comments that this is impossible from such a random collection. This data also does not line up with our goal of analyzing and inferring market sentiment of the TSLA stock, which is to be expected from an irrelevant subreddit. 

#### 5. Extract Top Level Comment from Subreddit `TSLA`.

Write your code to extract top level comments from the top 10 topics of a time period, e.g., year, from subreddit `TSLA` and store them in a list `top_comments_tsla`.  

In [57]:
subreddit_tsla = reddit.subreddit("TSLA")

top_comments_tsla = []

# Iteration through 10 top posts of all time in this subreddit
for submission in subreddit_tsla.top(time_filter="year", limit=10):
    # Iterate through top-level comments in the current post
    for top_level_comment in submission.comments:
        # Identify if this comment has more replies
        if isinstance(top_level_comment, MoreComments):
            continue
        # Add current comment to top comment list
        top_comments_tsla.append(top_level_comment.body)

In [58]:
len(top_comments_tsla) # Expected: 174 for r/machinelearning

158

In [61]:
[random.choice(top_comments_tsla) for i in range(3)]

['I remember when I first hit half a share pre split I was super excited. You’ll get to 1 in no time. Hopefully before the next split',
 'Bought 10 shares at $655!  It was a good time!',
 'About 98% for me 😁']

<details>
<summary>Some of the comments from `r/TSLA` subreddit:</summary>

    ['I bought puts',
    '100%',
    'Yes. And I’m bag holding 1200 calls for Friday and am close to throwing myself out the window']
</details>

💽❓ Data Question:

4. Now that you've had a chance to review another subreddits comments, do you see any differences in the kinds of comments either subreddit has - and how might this relate to bias?

ANSWER: Clearly, this subreddit is much more relevant to the TSLA stock and probably much more helpful in determing market sentiment. However, it can also be expected that a lot of the comments in this subreddit would be biased and not necessarily reflective of the market sentiment. One would expect that users who post in the TSLA subreddit probably hold TSLA stock, which would suggest that the amount of positive sentiment would be exaggerated, since a user would probably sell their TSLA stock if they felt negative sentiment towards it and stop following the TSLA subreddit. A more general subreddit, like r/stocks, might prove more useful in getting a less biased view of the TSLA stock, when filtering posts and comments for mentions of TSLA.

### Task III: Sentiment Analysis

Let us analyze the sentiment of comments scraped from `r/TSLA` using a pre-trained HuggingFace model to make the inference. Take a [Quick tour](https://huggingface.co/docs/transformers/quicktour). 

#### 1. Import `pipeline`

In [62]:
import transformers
from transformers import pipeline# YOUR CODE HERE

#### 2. Create a Pipeline to Perform Task "sentiment-analysis"

In [63]:
sentiment_model = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


#### 3. Get one comment from list `top_comments_tsla` from Task II - 5.

In [80]:
random.seed(42)
comment = random.choice(top_comments_tsla)

In [81]:
comment

"Jumped in for the first time at $805. I'm a happy camper :)"

The example comment is: `'Bury Burry!!!!!'`. Print out what you get. For reproducibility, use the same comment in the next step; consider setting a seed.

#### 4. Make Inference!

In [82]:
sentiment = sentiment_model(comment)
print(sentiment)

[{'label': 'POSITIVE', 'score': 0.9992936849594116}]


What is the type of the output `sentiment`?

In [88]:
print(type(sentiment))
print(type(sentiment[0]))

<class 'list'>
<class 'dict'>


```
ANSWER: The type of the sentiment is a list, which contains a single element, a dictionary with two elements, a label and a confidence score.
```

In [89]:
print(f'The comment: {comment}')
print(f'Predicted Label is {sentiment[0]["label"]} and the score is {sentiment[0]["score"]:.3f}')

The comment: Jumped in for the first time at $805. I'm a happy camper :)
Predicted Label is POSITIVE and the score is 0.999


For the example comment, the output is:

    The comment: Bury Burry!!!!!
    Predicted Label is NEGATIVE and the score is 0.989

🖥️❓ Model Question:

1. What does the score represent?

ANSWER: The score represents the confidence of the sentiment analysis model in making this classification ie. how "sure" the model is of its answer.

### Task IV: Put All Together

Let's pull all the piece together, create a simple script that does 

- get the subreddit
- get comments from the top posts for given subreddit
- run sentiment analysis 

#### Complete the Script

Once you complete the code, running the following block writes the code into a new Python script and saves it as `top_tlsa_comment_sentiment.py` under the same directory with the notebook. 

In [92]:
%%writefile top_tlsa_comment_sentiment.py

import reddit_secrets
import random

from typing import Dict, List

from praw import Reddit
from praw.models.reddit.subreddit import Subreddit
from praw.models import MoreComments

from transformers import pipeline


def get_subreddit(display_name:str) -> Subreddit:
    """Get subreddit object from display name

    Args:
        display_name (str): [description]

    Returns:
        Subreddit: [description]
    """
    reddit = Reddit(
        client_id=reddit_secrets.REDDIT_API_CLIENT_ID,        
        client_secret=reddit_secrets.REDDIT_API_CLIENT_SECRET,
        user_agent=reddit_secrets.REDDIT_API_USER_AGENT
        )
    
    subreddit = reddit.subreddit(display_name)
    return subreddit

def get_comments(subreddit:Subreddit, limit:int=3) -> List[str]:
    """ Get comments from subreddit

    Args:
        subreddit (Subreddit): [description]
        limit (int, optional): [description]. Defaults to 3.

    Returns:
        List[str]: List of comments
    """
    top_comments = []
    for submission in subreddit.top(limit=limit):
        for top_level_comment in submission.comments:
            if isinstance(top_level_comment, MoreComments):
                continue
            top_comments.append(top_level_comment.body)
    return top_comments

def run_sentiment_analysis(comment:str) -> Dict:
    """Run sentiment analysis on comment using default distilbert model
    
    Args:
        comment (str): [description]
        
    Returns:
        str: Sentiment analysis result
    """
    sentiment_model = pipeline("sentiment-analysis")
    sentiment = sentiment_model(comment)
    return sentiment[0]


if __name__ == '__main__':
    subreddit = get_subreddit("TSLA")
    comments = get_comments(subreddit)
    comment = random.choice(comments)
    sentiment = run_sentiment_analysis(comment)
    
    print(f'The comment: {comment}')
    print(f'Predicted Label is {sentiment["label"]} and the score is {sentiment["score"]:.3f}')

Overwriting top_tlsa_comment_sentiment.py


Run the following block to see the output.

In [93]:
!python top_tlsa_comment_sentiment.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
The comment: I can see a split after Elon sells his 10% as an announcement would drive up the price.
Predicted Label is NEGATIVE and the score is 0.998


<details><summary> Expected output:</summary>

    No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
    The comment: When is DOGE flying
    Predicted Label is POSITIVE and the score is 0.689
</details>

💽❓ Data Question:

5. Is the subreddit active? About how many posts or threads per day? How could you find this information?

In [95]:
counter = 0
for submission in subreddit_tsla.top(time_filter="day"):
    counter += 1
print(counter)

3


In [97]:
counter = 0
for submission in subreddit_tsla.top(time_filter="week"):
    counter += 1
print(round(counter/7))

2


ANSWER: The TSLA subreddit isn't that active when comparing it with large subreddits like r/machinelearning. In the past day there were only 3 posts. To get an idea of this number you could either use time_filter="day" in taking the top posts, with no limit set, or get an average over a period of time as demonstrated in the above two cell blocks.

💽❓ Data Question:

6. Does there seem to be a large distribution of posters or a smaller concentration of posters who are very active? What kind of impact might this have on the data?

In [111]:
user_dict = {}
for submission in subreddit_tsla.top(time_filter="year"):
    user_dict[submission.author] = user_dict.get(submission.author, 0) + 1

print(sorted(user_dict.values(), reverse=True))

[16, 9, 7, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


ANSWER: Despite there being quite a few users who have posted between 1 and 3 times on the subreddit, there is a smaller concentration of posters who are very active, particularly the posters who posted 7, 9, and 16 times respectively in the past year. Due to this small concentration of 3 relatively active posters, this would make the data biased, as the posts would reflect these few posters' sentiment rather than the market for the stock.